<a href="https://colab.research.google.com/github/kridtapon/EMA-Squeeze-Breakout/blob/main/EMA_Squeeze_Breakout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.6/527.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.5 MB/s eta 0:00:00


In [5]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Define the stock symbol and time period
symbol = 'META'  # SPY is the symbol for the S&P 500 ETF
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']

# Calculate 50 EMA and 200 EMA
df['50_EMA'] = df['Close'].ewm(span=50).mean()
df['200_EMA'] = df['Close'].ewm(span=200).mean()

# Calculate the distance between 50 EMA and 200 EMA (squeeze)
df['EMA_Distance'] = np.abs(df['50_EMA'] - df['200_EMA'])

# Define Squeeze Condition (when the distance between 50 EMA and 200 EMA is at a minimum)
squeeze_threshold = df['EMA_Distance'].rolling(window=20).min()
df['Squeeze'] = df['EMA_Distance'] <= squeeze_threshold

# Define Breakout Conditions
df['Breakout_Bullish'] = (df['Close'] > df['50_EMA']) & df['Squeeze']  # Bullish when price breaks above 50 EMA
df['Breakout_Bearish'] = (df['Close'] < df['200_EMA']) & df['Squeeze']  # Bearish when price breaks below 200 EMA

# Define Entry and Exit signals
df['Entry'] = df['Breakout_Bullish'] | df['Breakout_Bearish']
df['Exit'] = df['50_EMA'] < df['200_EMA']

# Filter data for the test period (2020-2025)
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=df['Entry'],
    exits=df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           259447.373558
Total Return [%]                       159.447374
Benchmark Return [%]                   180.172856
Max Gross Exposure [%]                      100.0
Total Fees Paid                        420.437077
Max Drawdown [%]                        30.213772
Max Drawdown Duration                       426.0
Total Trades                                    3
Total Closed Trades                             2
Total Open Trades                               1
Open Trade PnL                      173716.005917
Win Rate [%]                                 50.0
Best Trade [%]                          15.033617
Worst Trade [%]                        -25.488503
Avg Winning Trade [%]                   15.033617
Avg Losing Trade [%]                   -25.488503


In [5]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt
import itertools
import plotly.graph_objects as go

# Define the stock symbol and time period
symbol = 'TSLA'
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']

# Define parameter search space
ema_fast_values = range(10, 101, 5)  # Fast EMA range
ema_slow_values = range(100, 201, 5)  # Slow EMA range
squeeze_window_values = range(5, 51)  # Squeeze window range

best_return = -np.inf
best_params = None
results = []

# Iterate through all parameter combinations
for ema_fast, ema_slow, squeeze_window in itertools.product(ema_fast_values, ema_slow_values, squeeze_window_values):
    if ema_fast >= ema_slow:
        continue  # Skip invalid combinations

    df['Fast_EMA'] = df['Close'].ewm(span=ema_fast).mean()
    df['Slow_EMA'] = df['Close'].ewm(span=ema_slow).mean()
    df['EMA_Distance'] = np.abs(df['Fast_EMA'] - df['Slow_EMA'])

    squeeze_threshold = df['EMA_Distance'].rolling(window=squeeze_window).min()
    df['Squeeze'] = df['EMA_Distance'] <= squeeze_threshold

    df['Breakout_Bullish'] = (df['Close'] > df['Fast_EMA']) & df['Squeeze']
    df['Breakout_Bearish'] = (df['Close'] < df['Slow_EMA']) & df['Squeeze']
    df['Entry'] = df['Breakout_Bullish'] | df['Breakout_Bearish']
    df['Exit'] = df['Fast_EMA'] < df['Slow_EMA']

    # Filter data for the test period
    df_filtered = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

    portfolio = vbt.Portfolio.from_signals(
        close=df_filtered['Close'],
        entries=df_filtered['Entry'],
        exits=df_filtered['Exit'],
        init_cash=100_000,
        fees=0.001
    )

    total_return = portfolio.total_return()
    results.append((ema_fast, ema_slow, squeeze_window, total_return))

    if total_return > best_return:
        best_return = total_return
        best_params = (ema_fast, ema_slow, squeeze_window)

# Convert results to DataFrame
results_df = pd.DataFrame(results, columns=['EMA_Fast', 'EMA_Slow', 'Squeeze_Window', 'Total_Return'])

# 3D Plot
fig = go.Figure()
fig.add_trace(go.Scatter3d(
    x=results_df['EMA_Fast'],
    y=results_df['EMA_Slow'],
    z=results_df['Total_Return'],
    mode='markers',
    marker=dict(size=5, color=results_df['Total_Return'], colorscale='Viridis', opacity=0.8)
))
fig.update_layout(title='EMA Optimization Results', scene=dict(xaxis_title='EMA Fast', yaxis_title='EMA Slow', zaxis_title='Total Return'))
fig.show()

# Print best parameters
print(f'Best Parameters: EMA Fast = {best_params[0]}, EMA Slow = {best_params[1]}, Squeeze Window = {best_params[2]}')
print(f'Best Total Return: {best_return}')

[*********************100%***********************]  1 of 1 completed


Best Parameters: EMA Fast = 40, EMA Slow = 145, Squeeze Window = 28
Best Total Return: 17.992977166823174


In [6]:
# Calculate Fast EMA and Slow EMA
df['Fast_EMA'] = df['Close'].ewm(span=40).mean()
df['Slow_EMA'] = df['Close'].ewm(span=145).mean()

# Calculate the distance between Fast EMA and Slow EMA (squeeze)
df['EMA_Distance'] = np.abs(df['Fast_EMA'] - df['Slow_EMA'])

# Define Squeeze Condition (when the distance between Fast EMA and Slow EMA is at a minimum)
squeeze_threshold = df['EMA_Distance'].rolling(window=28).min()
df['Squeeze'] = df['EMA_Distance'] <= squeeze_threshold

# Define Breakout Conditions
df['Breakout_Bullish'] = (df['Close'] > df['Fast_EMA']) & df['Squeeze']  # Bullish when price breaks above Fast EMA
df['Breakout_Bearish'] = (df['Close'] < df['Slow_EMA']) & df['Squeeze']  # Bearish when price breaks below Slow EMA

# Define Entry and Exit signals
df['Entry'] = df['Breakout_Bullish'] | df['Breakout_Bearish']
df['Exit'] = df['Fast_EMA'] < df['Slow_EMA']

# Filter data for the test period (2020-2025)
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=df['Entry'],
    exits=df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                          1899297.716682
Total Return [%]                      1799.297717
Benchmark Return [%]                  1307.892888
Max Gross Exposure [%]                      100.0
Total Fees Paid                         8741.8289
Max Drawdown [%]                         46.39925
Max Drawdown Duration                       774.0
Total Trades                                    5
Total Closed Trades                             4
Total Open Trades                               1
Open Trade PnL                       891311.14377
Win Rate [%]                                 25.0
Best Trade [%]                        1059.112756
Worst Trade [%]                         -9.311172
Avg Winning Trade [%]                 1059.112756
Avg Losing Trade [%]                    -4.462336
